In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# Get Device for Training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [3]:
# Define the Class
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
# Using the model
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='mps:0')


In [6]:
# Model Layers
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2023, -0.1416, -0.0899, -0.0590, -0.1011, -0.4016, -0.1904,  0.2549,
          0.1741, -0.1188, -0.1359, -0.2196,  0.1863,  0.1602,  0.1990, -0.0983,
          0.6915,  0.0566,  0.4254,  0.4734],
        [ 0.0802, -0.3311,  0.3908, -0.1330, -0.2444, -0.0893, -0.7979,  0.2092,
          0.3510, -0.2564, -0.0150, -0.1060,  0.1622, -0.0596, -0.0655,  0.0595,
          0.2560,  0.2180,  0.0217,  0.4480],
        [-0.3232, -0.6133,  0.0485,  0.2278, -0.5668, -0.1866, -0.3176,  0.0703,
          0.3792, -0.3791,  0.0767,  0.0789,  0.2345,  0.2821, -0.1588, -0.1382,
          0.2343, -0.0307,  0.0294,  0.5486]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2549, 0.1741,
         0.0000, 0.0000, 0.0000, 0.1863, 0.1602, 0.1990, 0.0000, 0.6915, 0.0566,
         0.4254, 0.4734],
        [0.0802, 0.0000, 0.3908, 0.0000, 0.0000, 0.0000, 0.0000, 0.2092, 0.3510,
         0.0000, 0.0000, 0.0000, 0.1622, 0.0000, 0.00

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(f"logits: {logits}")

logits: tensor([[-0.3136, -0.2112, -0.1726, -0.2164, -0.2787, -0.0816, -0.1416, -0.0879,
         -0.1253, -0.3664],
        [-0.2292, -0.1236, -0.1205, -0.2478, -0.2556, -0.0035, -0.1667, -0.1469,
         -0.1597, -0.2497],
        [-0.2536, -0.1928, -0.1973, -0.1823, -0.2497, -0.0592, -0.1317, -0.1437,
         -0.1081, -0.2992]], grad_fn=<AddmmBackward0>)


In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(f"pred_probab: {pred_probab}")

pred_probab: tensor([[0.0888, 0.0984, 0.1023, 0.0979, 0.0920, 0.1121, 0.1055, 0.1114, 0.1073,
         0.0843],
        [0.0940, 0.1045, 0.1048, 0.0923, 0.0916, 0.1178, 0.1001, 0.1021, 0.1008,
         0.0921],
        [0.0928, 0.0987, 0.0982, 0.0997, 0.0932, 0.1128, 0.1049, 0.1036, 0.1074,
         0.0887]], grad_fn=<SoftmaxBackward0>)


In [12]:
# Model Parameters
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0104,  0.0277, -0.0308,  ..., -0.0178, -0.0260, -0.0224],
        [ 0.0100, -0.0266,  0.0271,  ..., -0.0289, -0.0114, -0.0162]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0228, -0.0329], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0292, -0.0214,  0.0123,  ...,  0.0324,  0.0287, -0.0215],
        [-0.0242,  0.0435, -0.0027,  ...,  0.0030,  0.0089, -0.0093]],
       device='mps:0', grad_fn=<Slice